# Deep Dive 03: Causal Forest 分裂准则深度解析

## 🎯 学习目标

完成本 notebook 后，你将能够：

1. **深入理解** Causal Forest 与普通随机森林的本质区别
2. **掌握** 基于 CATE 方差最大化的分裂准则
3. **实现** 从零开始构建 Causal Tree（不用 econml/grf）
4. **理解** Honest Splitting 和置信区间估计

---

## 📖 背景故事

> **场景：解释模型为什么这样分裂**
>
> 面试官问："你用过 Causal Forest，能告诉我它的分裂准则和 CART 有什么不同吗？"
>
> 你回答："嗯... 它是用来估计异质性效应的..."
>
> 面试官追问："具体怎么分裂？目标函数是什么？为什么要用 Honest Splitting？"
>
> 你："..."  😰
>
> **本 notebook 将带你彻底理解 Causal Forest 的核心原理！**

---

## Part 1: 从 CART 到 Causal Tree

### 1.1 普通 CART 的分裂准则

CART（Classification and Regression Tree）的分裂目标是**最小化子节点的不纯度**：

**回归树**：
$$\text{Impurity}(S) = \frac{1}{|S|} \sum_{i \in S} (Y_i - \bar{Y}_S)^2$$

分裂增益：
$$\text{Gain} = \text{Impurity}(S) - \left( \frac{|S_L|}{|S|} \text{Impurity}(S_L) + \frac{|S_R|}{|S|} \text{Impurity}(S_R) \right)$$

### 1.2 为什么 CART 不适合估计 CATE？

**问题**：CART 优化的是 $Y$ 的预测准确性，而不是处理效应的异质性！

考虑两种情况：

| 场景 | 结果 $Y$ 的方差 | CATE 的异质性 | CART 会选择？ |
|------|---------------|--------------|-------------|
| A | 高 | 低 | ✅ 会选择（减少 Y 方差） |
| B | 低 | 高 | ❌ 不会选择 |

**核心问题**：我们需要的分裂应该最大化 CATE 的异质性，而不是 Y 的方差！

In [ ]:
# 环境准备
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)

# 设置绘图风格
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['font.family'] = ['Arial Unicode MS', 'SimHei', 'sans-serif']
plt.rcParams['axes.unicode_minus'] = False

print("✅ 环境准备完成")

### 1.3 Causal Tree 的分裂准则

Causal Tree (Athey & Imbens, 2016) 的核心思想是：**最大化子节点间 CATE 的差异**

$$\text{CATE Gain} = \frac{|S_L||S_R|}{|S|^2} (\hat{\tau}(S_L) - \hat{\tau}(S_R))^2$$

其中 $\hat{\tau}(S)$ 是节点 $S$ 中的 CATE 估计：

$$\hat{\tau}(S) = \bar{Y}_{S,T=1} - \bar{Y}_{S,T=0}$$

**直觉**：我们希望找到一个分裂，使得左右子节点的处理效应差异最大化！

In [ ]:
# 可视化 CART vs Causal Tree 分裂的区别

def generate_example_data(n=1000):
    """生成展示两种分裂差异的数据"""
    # X1: 影响 Y 的基础水平（CART 会关注）
    # X2: 影响处理效应（Causal Tree 应该关注）
    X1 = np.random.randn(n)
    X2 = np.random.randn(n)
    T = np.random.binomial(1, 0.5, n)
    
    # Y 的基础值主要由 X1 决定
    Y_base = 10 + 5 * X1  # X1 影响基础水平
    
    # 处理效应主要由 X2 决定
    tau = 3 + 4 * X2  # X2 影响处理效应
    
    # 观测结果
    Y = Y_base + T * tau + np.random.randn(n) * 2
    
    return X1, X2, T, Y, tau

X1, X2, T, Y, true_tau = generate_example_data(2000)

# 可视化
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# X1 与 Y 的关系（高方差）
ax1 = axes[0]
ax1.scatter(X1[T==0], Y[T==0], alpha=0.3, label='控制组', c='#3498db', s=20)
ax1.scatter(X1[T==1], Y[T==1], alpha=0.3, label='实验组', c='#e74c3c', s=20)
ax1.set_xlabel('X1')
ax1.set_ylabel('Y')
ax1.set_title('X1 影响 Y 的基础水平（高方差）\nCART 会优先选择这个变量')
ax1.legend()

# X2 与 CATE 的关系
ax2 = axes[1]
ax2.scatter(X2, true_tau, alpha=0.5, c='#2ecc71', s=20)
ax2.set_xlabel('X2')
ax2.set_ylabel('CATE')
ax2.set_title('X2 影响处理效应（CATE 异质性）\nCausal Tree 应该选择这个变量')

# 两种分裂的对比
ax3 = axes[2]
# 按 X1 分裂
mask_x1_low = X1 < np.median(X1)
tau_x1_left = Y[mask_x1_low & (T==1)].mean() - Y[mask_x1_low & (T==0)].mean()
tau_x1_right = Y[~mask_x1_low & (T==1)].mean() - Y[~mask_x1_low & (T==0)].mean()

# 按 X2 分裂
mask_x2_low = X2 < np.median(X2)
tau_x2_left = Y[mask_x2_low & (T==1)].mean() - Y[mask_x2_low & (T==0)].mean()
tau_x2_right = Y[~mask_x2_low & (T==1)].mean() - Y[~mask_x2_low & (T==0)].mean()

x_pos = [0, 1]
width = 0.35

ax3.bar([p - width/2 for p in x_pos], [tau_x1_left, tau_x2_left], width, label='左子节点 CATE', color='#3498db')
ax3.bar([p + width/2 for p in x_pos], [tau_x1_right, tau_x2_right], width, label='右子节点 CATE', color='#e74c3c')
ax3.set_xticks(x_pos)
ax3.set_xticklabels(['按 X1 分裂', '按 X2 分裂'])
ax3.set_ylabel('CATE')
ax3.set_title('两种分裂的 CATE 差异')
ax3.legend()

# 标注 CATE 差异
diff_x1 = abs(tau_x1_right - tau_x1_left)
diff_x2 = abs(tau_x2_right - tau_x2_left)
ax3.text(-0.15, max(tau_x1_left, tau_x1_right) + 0.5, f'差异: {diff_x1:.2f}', fontsize=10)
ax3.text(0.85, max(tau_x2_left, tau_x2_right) + 0.5, f'差异: {diff_x2:.2f}', fontsize=10)

plt.tight_layout()
plt.show()

print(f"\n关键发现：")
print(f"  按 X1 分裂的 CATE 差异: {diff_x1:.3f}")
print(f"  按 X2 分裂的 CATE 差异: {diff_x2:.3f}")
print(f"  ✅ Causal Tree 应该选择 X2，因为它最大化了 CATE 差异！")

---

## Part 2: 从零实现 Causal Tree 分裂准则

### 🧪 TODO 练习 1: 实现 CATE 增益计算

请实现 Causal Tree 的分裂增益计算函数。

In [ ]:
def calculate_cate_gain(Y_left, T_left, Y_right, T_right, min_samples=10):
    """
    计算 Causal Tree 分裂增益
    
    CATE Gain = (n_L * n_R / n^2) * (tau_L - tau_R)^2
    
    Args:
        Y_left: 左子节点的结果
        T_left: 左子节点的处理
        Y_right: 右子节点的结果
        T_right: 右子节点的处理
        min_samples: 最小样本量阈值
        
    Returns:
        gain: CATE 分裂增益
        tau_left: 左子节点 CATE
        tau_right: 右子节点 CATE
    """
    # 实现 CATE 增益计算
    n_left = len(Y_left)
    n_right = len(Y_right)
    n_total = n_left + n_right
    
    # 检查每个子节点的处理组和控制组样本量
    n_treat_left = (T_left == 1).sum()
    n_ctrl_left = (T_left == 0).sum()
    n_treat_right = (T_right == 1).sum()
    n_ctrl_right = (T_right == 0).sum()
    
    # 确保每个子节点有足够的处理组和控制组样本
    if (n_treat_left < min_samples or n_ctrl_left < min_samples or
        n_treat_right < min_samples or n_ctrl_right < min_samples):
        return 0, np.nan, np.nan
    
    # 1. 计算左子节点的 CATE: tau_left = mean(Y|T=1, left) - mean(Y|T=0, left)
    tau_left = Y_left[T_left == 1].mean() - Y_left[T_left == 0].mean()
    
    # 2. 计算右子节点的 CATE: tau_right = mean(Y|T=1, right) - mean(Y|T=0, right)
    tau_right = Y_right[T_right == 1].mean() - Y_right[T_right == 0].mean()
    
    # 3. 计算增益: gain = (n_L * n_R / n^2) * (tau_left - tau_right)^2
    gain = (n_left * n_right / (n_total ** 2)) * (tau_left - tau_right) ** 2
    
    return gain, tau_left, tau_right

print("✅ calculate_cate_gain 函数定义完成（已完成 TODO 部分）")

### 📝 参考答案

In [ ]:
# ============================================================
# 参考答案：CATE 增益计算
# ============================================================

def calculate_cate_gain(Y_left, T_left, Y_right, T_right, min_samples=10):
    """
    计算 Causal Tree 分裂增益 - 完整实现
    """
    n_left = len(Y_left)
    n_right = len(Y_right)
    n_total = n_left + n_right
    
    # 检查每个子节点的处理组和控制组样本量
    n_treat_left = (T_left == 1).sum()
    n_ctrl_left = (T_left == 0).sum()
    n_treat_right = (T_right == 1).sum()
    n_ctrl_right = (T_right == 0).sum()
    
    # 确保每个子节点有足够的处理组和控制组样本
    if (n_treat_left < min_samples or n_ctrl_left < min_samples or
        n_treat_right < min_samples or n_ctrl_right < min_samples):
        return 0, np.nan, np.nan
    
    # 计算左子节点 CATE
    tau_left = Y_left[T_left == 1].mean() - Y_left[T_left == 0].mean()
    
    # 计算右子节点 CATE
    tau_right = Y_right[T_right == 1].mean() - Y_right[T_right == 0].mean()
    
    # 计算增益
    gain = (n_left * n_right / (n_total ** 2)) * (tau_left - tau_right) ** 2
    
    return gain, tau_left, tau_right

# 测试
np.random.seed(42)
n_test = 500
Y_test = np.random.randn(n_test) + 10
T_test = np.random.binomial(1, 0.5, n_test)

# 人为构造左右子节点
Y_left = Y_test[:n_test//2]
T_left = T_test[:n_test//2]
Y_right = Y_test[n_test//2:] + 3 * T_test[n_test//2:]  # 右边加入处理效应
T_right = T_test[n_test//2:]

gain, tau_l, tau_r = calculate_cate_gain(Y_left, T_left, Y_right, T_right)
print(f"✅ CATE 增益计算测试:")
print(f"   左子节点 CATE: {tau_l:.3f}")
print(f"   右子节点 CATE: {tau_r:.3f}")
print(f"   分裂增益: {gain:.4f}")

### 2.1 实现完整的 Causal Tree

In [ ]:
class CausalTree:
    """
    从零实现的 Causal Tree
    分裂准则：最大化子节点间的 CATE 差异
    """
    def __init__(self, max_depth=5, min_samples_leaf=20, min_samples_treatment=10):
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.min_samples_treatment = min_samples_treatment
        self.tree = None
    
    def _calculate_cate(self, Y, T):
        """计算节点的 CATE"""
        if (T == 1).sum() == 0 or (T == 0).sum() == 0:
            return np.nan
        return Y[T == 1].mean() - Y[T == 0].mean()
    
    def _find_best_split(self, X, Y, T):
        """
        找到最佳分裂点
        遍历所有特征和所有分裂点，选择 CATE 增益最大的
        """
        n_samples, n_features = X.shape
        best_gain = 0
        best_feature = None
        best_threshold = None
        
        for feature_idx in range(n_features):
            # 获取所有可能的分裂点
            thresholds = np.unique(X[:, feature_idx])
            
            for threshold in thresholds[:-1]:  # 排除最大值
                # 分裂
                left_mask = X[:, feature_idx] <= threshold
                right_mask = ~left_mask
                
                # 检查样本量
                if left_mask.sum() < self.min_samples_leaf or right_mask.sum() < self.min_samples_leaf:
                    continue
                
                # 计算 CATE 增益
                gain, _, _ = calculate_cate_gain(
                    Y[left_mask], T[left_mask],
                    Y[right_mask], T[right_mask],
                    min_samples=self.min_samples_treatment
                )
                
                if gain > best_gain:
                    best_gain = gain
                    best_feature = feature_idx
                    best_threshold = threshold
        
        return best_feature, best_threshold, best_gain
    
    def _build_tree(self, X, Y, T, depth=0):
        """
        递归构建树
        """
        n_samples = len(Y)
        node_cate = self._calculate_cate(Y, T)
        
        node = {
            'cate': node_cate,
            'n_samples': n_samples,
            'n_treated': (T == 1).sum(),
            'n_control': (T == 0).sum()
        }
        
        # 停止条件
        if (depth >= self.max_depth or 
            n_samples < 2 * self.min_samples_leaf or
            (T == 1).sum() < 2 * self.min_samples_treatment or
            (T == 0).sum() < 2 * self.min_samples_treatment):
            return node
        
        # 找最佳分裂
        best_feature, best_threshold, best_gain = self._find_best_split(X, Y, T)
        
        if best_feature is None or best_gain == 0:
            return node
        
        # 执行分裂
        left_mask = X[:, best_feature] <= best_threshold
        right_mask = ~left_mask
        
        node['feature'] = best_feature
        node['threshold'] = best_threshold
        node['gain'] = best_gain
        node['left'] = self._build_tree(X[left_mask], Y[left_mask], T[left_mask], depth + 1)
        node['right'] = self._build_tree(X[right_mask], Y[right_mask], T[right_mask], depth + 1)
        
        return node
    
    def fit(self, X, Y, T):
        """训练 Causal Tree"""
        X = np.array(X)
        Y = np.array(Y)
        T = np.array(T)
        self.tree = self._build_tree(X, Y, T)
        return self
    
    def _predict_single(self, x, node):
        """预测单个样本的 CATE"""
        if 'feature' not in node:
            return node['cate']
        
        if x[node['feature']] <= node['threshold']:
            return self._predict_single(x, node['left'])
        else:
            return self._predict_single(x, node['right'])
    
    def predict(self, X):
        """预测 CATE"""
        X = np.array(X)
        return np.array([self._predict_single(x, self.tree) for x in X])
    
    def print_tree(self, node=None, depth=0, prefix=""):
        """打印树结构"""
        if node is None:
            node = self.tree
        
        if 'feature' not in node:
            print(f"{prefix}Leaf: CATE={node['cate']:.3f}, n={node['n_samples']}")
        else:
            print(f"{prefix}X[{node['feature']}] <= {node['threshold']:.3f} (gain={node['gain']:.4f})")
            self.print_tree(node['left'], depth + 1, prefix + "  L: ")
            self.print_tree(node['right'], depth + 1, prefix + "  R: ")

print("✅ CausalTree 类定义完成")

In [ ]:
# 训练和测试 Causal Tree

def generate_causal_data(n=2000, seed=42):
    """
    生成带有异质性处理效应的数据
    
    真实 CATE = 2 + 3*X1 + 4*X2 + 2*X1*X2
    """
    np.random.seed(seed)
    
    X = np.random.randn(n, 5)  # 5 个特征
    T = np.random.binomial(1, 0.5, n)  # 随机处理分配
    
    # 真实 CATE（只依赖 X0 和 X1）
    true_cate = 2 + 3 * X[:, 0] + 4 * X[:, 1] + 2 * X[:, 0] * X[:, 1]
    
    # 基础结果（依赖所有特征）
    Y_base = 10 + 2*X[:, 0] + 1*X[:, 1] + 0.5*X[:, 2] + 0.3*X[:, 3] + 0.2*X[:, 4]
    
    # 观测结果
    noise = np.random.randn(n) * 2
    Y = Y_base + T * true_cate + noise
    
    return X, T, Y, true_cate

# 生成数据
X, T, Y, true_cate = generate_causal_data(n=2000)

print(f"数据生成完成：")
print(f"  样本数: {len(Y)}")
print(f"  特征数: {X.shape[1]}")
print(f"  处理组: {(T==1).sum()}")
print(f"  真实 CATE 范围: [{true_cate.min():.2f}, {true_cate.max():.2f}]")

In [ ]:
# 训练 Causal Tree
ct = CausalTree(max_depth=3, min_samples_leaf=50, min_samples_treatment=20)
ct.fit(X, Y, T)

print("Causal Tree 结构：")
print("=" * 50)
ct.print_tree()

In [ ]:
# 评估 CATE 预测
pred_cate = ct.predict(X)

# 计算相关性
corr = np.corrcoef(pred_cate, true_cate)[0, 1]
rmse = np.sqrt(np.mean((pred_cate - true_cate) ** 2))

print(f"\nCausal Tree 评估结果：")
print(f"  相关系数: {corr:.3f}")
print(f"  RMSE: {rmse:.3f}")

# 可视化
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 预测 vs 真实
ax1 = axes[0]
ax1.scatter(true_cate, pred_cate, alpha=0.3, s=20)
ax1.plot([true_cate.min(), true_cate.max()], [true_cate.min(), true_cate.max()], 
         'r--', label='完美预测')
ax1.set_xlabel('真实 CATE')
ax1.set_ylabel('预测 CATE')
ax1.set_title(f'Causal Tree CATE 预测\nCorr={corr:.3f}, RMSE={rmse:.3f}')
ax1.legend()

# 按 X0 分组看 CATE
ax2 = axes[1]
x0_bins = pd.qcut(X[:, 0], q=5, labels=['Q1', 'Q2', 'Q3', 'Q4', 'Q5'])
df = pd.DataFrame({'X0_bin': x0_bins, 'true_cate': true_cate, 'pred_cate': pred_cate})
df_grouped = df.groupby('X0_bin').agg({'true_cate': 'mean', 'pred_cate': 'mean'}).reset_index()

x_pos = np.arange(5)
width = 0.35
ax2.bar(x_pos - width/2, df_grouped['true_cate'], width, label='真实 CATE', color='#3498db')
ax2.bar(x_pos + width/2, df_grouped['pred_cate'], width, label='预测 CATE', color='#e74c3c')
ax2.set_xticks(x_pos)
ax2.set_xticklabels(df_grouped['X0_bin'])
ax2.set_xlabel('X0 分位数')
ax2.set_ylabel('平均 CATE')
ax2.set_title('按 X0 分组的 CATE 对比')
ax2.legend()

plt.tight_layout()
plt.show()

---

## Part 3: Honest Splitting 详解

### 3.1 为什么需要 Honest Splitting？

**问题**：如果用同一份数据既选择分裂点又估计 CATE，会导致**过拟合**！

| 问题 | 描述 |
|------|------|
| 选择偏差 | 分裂点是根据数据"优化"出来的，会过拟合噪声 |
| 估计偏差 | 叶节点的 CATE 估计使用了参与分裂决策的样本 |
| 置信区间失效 | 无法正确计算标准误 |

### 3.2 Honest Splitting 方案

**核心思想**：将数据分成两部分
1. **Splitting Sample**: 用于决定分裂点
2. **Estimation Sample**: 用于估计叶节点的 CATE

```
数据 D
   │
   ├── Splitting Sample (D_s) ──→ 决定树结构
   │
   └── Estimation Sample (D_e) ──→ 估计叶节点 CATE + 置信区间
```

In [ ]:
class HonestCausalTree:
    """
    Honest Causal Tree
    分裂和估计使用不同的样本
    """
    def __init__(self, max_depth=5, min_samples_leaf=20, min_samples_treatment=10, 
                 honest_fraction=0.5):
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.min_samples_treatment = min_samples_treatment
        self.honest_fraction = honest_fraction
        self.tree = None
        self.estimation_indices = None
    
    def _calculate_cate_with_se(self, Y, T):
        """
        计算 CATE 和标准误
        """
        n_t = (T == 1).sum()
        n_c = (T == 0).sum()
        
        if n_t < 2 or n_c < 2:
            return np.nan, np.nan
        
        Y_t = Y[T == 1]
        Y_c = Y[T == 0]
        
        tau = Y_t.mean() - Y_c.mean()
        se = np.sqrt(np.var(Y_t, ddof=1)/n_t + np.var(Y_c, ddof=1)/n_c)
        
        return tau, se
    
    def fit(self, X, Y, T):
        """
        训练 Honest Causal Tree
        """
        X = np.array(X)
        Y = np.array(Y)
        T = np.array(T)
        n = len(Y)
        
        # 分割 Splitting 和 Estimation 样本
        indices = np.random.permutation(n)
        n_split = int(n * (1 - self.honest_fraction))
        
        split_idx = indices[:n_split]
        est_idx = indices[n_split:]
        
        X_split, Y_split, T_split = X[split_idx], Y[split_idx], T[split_idx]
        self.X_est, self.Y_est, self.T_est = X[est_idx], Y[est_idx], T[est_idx]
        
        # 用 Splitting Sample 构建树结构
        base_tree = CausalTree(
            max_depth=self.max_depth,
            min_samples_leaf=self.min_samples_leaf,
            min_samples_treatment=self.min_samples_treatment
        )
        base_tree.fit(X_split, Y_split, T_split)
        self.tree = base_tree.tree
        
        # 用 Estimation Sample 重新估计叶节点
        self._update_leaf_estimates()
        
        return self
    
    def _get_leaf_mask(self, X, node):
        """
        获取属于某个叶节点的样本 mask
        """
        if 'feature' not in node:
            return np.ones(len(X), dtype=bool)
        
        left_mask = X[:, node['feature']] <= node['threshold']
        
        left_child_mask = self._get_leaf_mask(X[left_mask], node['left'])
        right_child_mask = self._get_leaf_mask(X[~left_mask], node['right'])
        
        result = np.zeros(len(X), dtype=bool)
        result[np.where(left_mask)[0][left_child_mask]] = True
        result[np.where(~left_mask)[0][right_child_mask]] = True
        
        return result
    
    def _update_leaf_node(self, node, X, Y, T):
        """
        递归更新叶节点的 CATE 和标准误
        """
        if 'feature' not in node:
            # 叶节点：使用 Estimation Sample 重新估计
            tau, se = self._calculate_cate_with_se(Y, T)
            node['cate'] = tau
            node['se'] = se
            node['n_samples'] = len(Y)
            node['n_treated'] = (T == 1).sum()
            node['n_control'] = (T == 0).sum()
            return
        
        # 内部节点：递归更新子节点
        left_mask = X[:, node['feature']] <= node['threshold']
        self._update_leaf_node(node['left'], X[left_mask], Y[left_mask], T[left_mask])
        self._update_leaf_node(node['right'], X[~left_mask], Y[~left_mask], T[~left_mask])
    
    def _update_leaf_estimates(self):
        """使用 Estimation Sample 更新所有叶节点"""
        self._update_leaf_node(self.tree, self.X_est, self.Y_est, self.T_est)
    
    def _predict_single(self, x, node):
        """预测单个样本"""
        if 'feature' not in node:
            return node['cate'], node['se']
        
        if x[node['feature']] <= node['threshold']:
            return self._predict_single(x, node['left'])
        else:
            return self._predict_single(x, node['right'])
    
    def predict(self, X, return_ci=False, alpha=0.05):
        """
        预测 CATE，可选返回置信区间
        """
        X = np.array(X)
        predictions = np.array([self._predict_single(x, self.tree) for x in X])
        
        cate = predictions[:, 0]
        se = predictions[:, 1]
        
        if return_ci:
            z = stats.norm.ppf(1 - alpha/2)
            ci_lower = cate - z * se
            ci_upper = cate + z * se
            return cate, ci_lower, ci_upper
        
        return cate

print("✅ HonestCausalTree 类定义完成")

In [ ]:
# 训练 Honest Causal Tree
hct = HonestCausalTree(max_depth=3, min_samples_leaf=50, honest_fraction=0.5)
hct.fit(X, Y, T)

# 预测（带置信区间）
pred_cate_honest, ci_lower, ci_upper = hct.predict(X, return_ci=True)

# 评估
corr_honest = np.corrcoef(pred_cate_honest, true_cate)[0, 1]
rmse_honest = np.sqrt(np.mean((pred_cate_honest - true_cate) ** 2))

# 置信区间覆盖率
coverage = np.mean((true_cate >= ci_lower) & (true_cate <= ci_upper))

print(f"\nHonest Causal Tree 评估结果：")
print(f"  相关系数: {corr_honest:.3f}")
print(f"  RMSE: {rmse_honest:.3f}")
print(f"  95% CI 覆盖率: {coverage:.1%} (目标: 95%)")

In [ ]:
# 可视化 Honest vs Non-Honest 对比
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# 预测对比
ax1 = axes[0]
ax1.scatter(true_cate, pred_cate, alpha=0.3, label='普通 Causal Tree', s=20)
ax1.scatter(true_cate, pred_cate_honest, alpha=0.3, label='Honest Causal Tree', s=20)
ax1.plot([true_cate.min(), true_cate.max()], [true_cate.min(), true_cate.max()], 'k--')
ax1.set_xlabel('真实 CATE')
ax1.set_ylabel('预测 CATE')
ax1.set_title('CATE 预测对比')
ax1.legend()

# 置信区间可视化
ax2 = axes[1]
# 选取部分样本展示
idx_sample = np.random.choice(len(X), 50, replace=False)
idx_sorted = idx_sample[np.argsort(true_cate[idx_sample])]

ax2.errorbar(range(50), pred_cate_honest[idx_sorted], 
            yerr=[pred_cate_honest[idx_sorted] - ci_lower[idx_sorted],
                  ci_upper[idx_sorted] - pred_cate_honest[idx_sorted]],
            fmt='o', capsize=3, alpha=0.6, label='预测 ± 95% CI')
ax2.scatter(range(50), true_cate[idx_sorted], color='red', s=30, zorder=5, label='真实 CATE')
ax2.set_xlabel('样本（按真实 CATE 排序）')
ax2.set_ylabel('CATE')
ax2.set_title('置信区间覆盖示意')
ax2.legend()

# 覆盖率检验
ax3 = axes[2]
# 模拟多次检验覆盖率
coverages = []
for _ in range(100):
    hct_temp = HonestCausalTree(max_depth=3, min_samples_leaf=50, honest_fraction=0.5)
    hct_temp.fit(X, Y, T)
    _, ci_l, ci_u = hct_temp.predict(X, return_ci=True)
    cov = np.mean((true_cate >= ci_l) & (true_cate <= ci_u))
    coverages.append(cov)

ax3.hist(coverages, bins=15, alpha=0.7, color='#3498db', edgecolor='black')
ax3.axvline(x=0.95, color='red', linestyle='--', linewidth=2, label='目标 95%')
ax3.axvline(x=np.mean(coverages), color='green', linestyle='-', linewidth=2, 
           label=f'平均 {np.mean(coverages):.1%}')
ax3.set_xlabel('95% CI 覆盖率')
ax3.set_ylabel('频数')
ax3.set_title('100 次模拟的覆盖率分布')
ax3.legend()

plt.tight_layout()
plt.show()

---

## Part 4: Causal Forest 实现

### 4.1 从 Causal Tree 到 Causal Forest

Causal Forest 是 Causal Tree 的集成版本，通过 Bootstrap 聚合提高稳定性。

**关键特点**：
1. 每棵树使用 Bootstrap 样本
2. 每次分裂随机选择特征子集
3. 使用 Honest Splitting
4. 最终预测是所有树的平均

In [ ]:
class SimpleCausalForest:
    """
    简化版 Causal Forest
    """
    def __init__(self, n_trees=100, max_depth=5, min_samples_leaf=20,
                 honest_fraction=0.5, max_features='sqrt'):
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.honest_fraction = honest_fraction
        self.max_features = max_features
        self.trees = []
    
    def fit(self, X, Y, T):
        """
        训练 Causal Forest
        """
        X = np.array(X)
        Y = np.array(Y)
        T = np.array(T)
        n_samples, n_features = X.shape
        
        # 确定每次分裂的特征数
        if self.max_features == 'sqrt':
            self.n_features_split = int(np.sqrt(n_features))
        elif self.max_features == 'log2':
            self.n_features_split = int(np.log2(n_features))
        else:
            self.n_features_split = n_features
        
        self.trees = []
        
        for i in range(self.n_trees):
            # Bootstrap 采样
            boot_idx = np.random.choice(n_samples, n_samples, replace=True)
            X_boot = X[boot_idx]
            Y_boot = Y[boot_idx]
            T_boot = T[boot_idx]
            
            # 随机选择特征子集
            feature_idx = np.random.choice(n_features, self.n_features_split, replace=False)
            X_subset = X_boot[:, feature_idx]
            
            # 训练 Honest Causal Tree
            tree = HonestCausalTree(
                max_depth=self.max_depth,
                min_samples_leaf=self.min_samples_leaf,
                honest_fraction=self.honest_fraction
            )
            tree.fit(X_subset, Y_boot, T_boot)
            
            self.trees.append((tree, feature_idx))
        
        return self
    
    def predict(self, X, return_ci=False, alpha=0.05):
        """
        预测 CATE
        
        使用所有树预测的平均值和标准差来估计置信区间
        """
        X = np.array(X)
        all_predictions = []
        
        for tree, feature_idx in self.trees:
            X_subset = X[:, feature_idx]
            pred = tree.predict(X_subset)
            all_predictions.append(pred)
        
        all_predictions = np.array(all_predictions)
        
        # 平均预测
        cate_mean = np.mean(all_predictions, axis=0)
        
        if return_ci:
            # 使用树之间的方差估计置信区间
            cate_se = np.std(all_predictions, axis=0) / np.sqrt(self.n_trees)
            z = stats.norm.ppf(1 - alpha/2)
            ci_lower = cate_mean - z * cate_se
            ci_upper = cate_mean + z * cate_se
            return cate_mean, ci_lower, ci_upper
        
        return cate_mean

print("✅ SimpleCausalForest 类定义完成")

In [ ]:
# 训练 Causal Forest
print("训练 Causal Forest (这可能需要一些时间)...")
cf = SimpleCausalForest(n_trees=50, max_depth=4, min_samples_leaf=30)
cf.fit(X, Y, T)
print("✅ 训练完成")

# 预测
pred_cate_cf, ci_lower_cf, ci_upper_cf = cf.predict(X, return_ci=True)

# 评估
corr_cf = np.corrcoef(pred_cate_cf, true_cate)[0, 1]
rmse_cf = np.sqrt(np.mean((pred_cate_cf - true_cate) ** 2))
coverage_cf = np.mean((true_cate >= ci_lower_cf) & (true_cate <= ci_upper_cf))

print(f"\nCausal Forest 评估结果：")
print(f"  相关系数: {corr_cf:.3f}")
print(f"  RMSE: {rmse_cf:.3f}")
print(f"  95% CI 覆盖率: {coverage_cf:.1%}")

In [ ]:
# 三种方法对比
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

methods = ['Causal Tree', 'Honest Causal Tree', 'Causal Forest']
predictions = [pred_cate, pred_cate_honest, pred_cate_cf]
colors = ['#3498db', '#e74c3c', '#2ecc71']

for i, (method, pred, color) in enumerate(zip(methods, predictions, colors)):
    ax = axes[i]
    ax.scatter(true_cate, pred, alpha=0.3, s=20, c=color)
    ax.plot([true_cate.min(), true_cate.max()], [true_cate.min(), true_cate.max()], 'k--')
    
    corr = np.corrcoef(pred, true_cate)[0, 1]
    rmse = np.sqrt(np.mean((pred - true_cate) ** 2))
    
    ax.set_xlabel('真实 CATE')
    ax.set_ylabel('预测 CATE')
    ax.set_title(f'{method}\nCorr={corr:.3f}, RMSE={rmse:.3f}')

plt.tight_layout()
plt.show()

# 汇总表格
print("\n" + "=" * 60)
print("方法对比汇总")
print("=" * 60)
print(f"\n{'方法':<25} {'相关系数':<12} {'RMSE':<12}")
print("-" * 50)
print(f"{'Causal Tree':<25} {corr:.3f}        {rmse:.3f}")
print(f"{'Honest Causal Tree':<25} {corr_honest:.3f}        {rmse_honest:.3f}")
print(f"{'Causal Forest':<25} {corr_cf:.3f}        {rmse_cf:.3f}")

---

## Part 5: 与 EconML 的 CausalForest 对比

让我们将自己实现的版本与 EconML 的实现进行对比。

In [ ]:
try:
    from econml.dml import CausalForestDML
    
    # 训练 EconML 的 Causal Forest
    print("训练 EconML CausalForestDML...")
    cf_econml = CausalForestDML(
        n_estimators=100,
        max_depth=4,
        min_samples_leaf=30,
        random_state=42
    )
    cf_econml.fit(Y, T, X=X, W=None)
    
    # 预测
    pred_econml = cf_econml.effect(X)
    pred_econml_lb, pred_econml_ub = cf_econml.effect_interval(X, alpha=0.05)
    
    # 评估
    corr_econml = np.corrcoef(pred_econml.flatten(), true_cate)[0, 1]
    rmse_econml = np.sqrt(np.mean((pred_econml.flatten() - true_cate) ** 2))
    coverage_econml = np.mean((true_cate >= pred_econml_lb.flatten()) & 
                               (true_cate <= pred_econml_ub.flatten()))
    
    print(f"\nEconML CausalForestDML 评估结果：")
    print(f"  相关系数: {corr_econml:.3f}")
    print(f"  RMSE: {rmse_econml:.3f}")
    print(f"  95% CI 覆盖率: {coverage_econml:.1%}")
    
    # 对比可视化
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    ax1 = axes[0]
    ax1.scatter(true_cate, pred_cate_cf, alpha=0.3, label='自实现', s=20)
    ax1.scatter(true_cate, pred_econml.flatten(), alpha=0.3, label='EconML', s=20)
    ax1.plot([true_cate.min(), true_cate.max()], [true_cate.min(), true_cate.max()], 'k--')
    ax1.set_xlabel('真实 CATE')
    ax1.set_ylabel('预测 CATE')
    ax1.set_title('自实现 vs EconML')
    ax1.legend()
    
    ax2 = axes[1]
    methods = ['自实现 CF', 'EconML CF']
    corrs = [corr_cf, corr_econml]
    rmses = [rmse_cf, rmse_econml]
    
    x = np.arange(2)
    width = 0.35
    ax2.bar(x - width/2, corrs, width, label='相关系数', color='#3498db')
    ax2.bar(x + width/2, [r/10 for r in rmses], width, label='RMSE/10', color='#e74c3c')
    ax2.set_xticks(x)
    ax2.set_xticklabels(methods)
    ax2.set_ylabel('值')
    ax2.set_title('性能指标对比')
    ax2.legend()
    
    plt.tight_layout()
    plt.show()
    
except ImportError:
    print("⚠️ EconML 未安装，跳过对比")
    print("   安装命令: pip install econml")

---

## 💡 思考题

1. **分裂准则变体**：除了最大化 CATE 差异，还有什么其他分裂准则？
   - 提示：考虑 CATE 的方差、MSE 等

2. **权重调整**：如果处理组和控制组样本不平衡，分裂准则应该如何调整？
   - 提示：考虑 IPW 权重

3. **置信区间校准**：如果置信区间覆盖率不足 95%，如何改进？
   - 提示：Bootstrap、交叉验证

4. **特征重要性**：如何计算 Causal Forest 中的特征重要性？
   - 提示：基于 CATE 增益的累积

---

## 📝 总结

### 核心概念

| 概念 | 描述 |
|------|------|
| CATE 增益 | (n_L * n_R / n²) * (τ_L - τ_R)² |
| Honest Splitting | 分裂和估计使用不同样本 |
| Causal Forest | Causal Tree + Bootstrap + 特征随机 |

### CART vs Causal Tree 对比

| 维度 | CART | Causal Tree |
|------|------|-------------|
| 优化目标 | 最小化 Y 的方差 | 最大化 CATE 差异 |
| 叶节点输出 | E[Y] | τ = E[Y(1) - Y(0)] |
| 适用场景 | 预测 | 因果效应估计 |

### 面试要点

- 能解释 Causal Tree 与 CART 的本质区别
- 能写出 CATE 增益的公式
- 理解 Honest Splitting 的必要性
- 知道如何输出置信区间

---

## 🔗 参考资料

1. Athey, S., & Imbens, G. W. (2016). Recursive Partitioning for Heterogeneous Causal Effects. PNAS.
2. Wager, S., & Athey, S. (2018). Estimation and Inference of Heterogeneous Treatment Effects using Random Forests. JASA.
3. EconML Documentation: https://econml.azurewebsites.net/
4. GRF (Generalized Random Forests): https://grf-labs.github.io/grf/

---

## 🎤 面试模拟环节

### 问题 1: CART vs Causal Tree - 本质区别

**面试官**: CART 和 Causal Tree 的分裂准则有什么本质区别？为什么不能直接用 CART 估计 CATE？

**参考答案**:

**核心区别**:

| 维度 | CART | Causal Tree |
|------|------|-------------|
| **优化目标** | 最小化 Y 的方差 | 最大化 CATE 的异质性 |
| **分裂增益** | $\Delta \text{Var}(Y)$ | $\frac{n_L n_R}{n^2}(\tau_L - \tau_R)^2$ |
| **叶节点输出** | $\bar{Y}$ | $\tau = \bar{Y}_1 - \bar{Y}_0$ |

**为什么 CART 不适合**:

```python
# 场景：两个候选分裂
# 分裂A: 减少 Y 方差很多，但 CATE 无差异
# 分裂B: Y 方差不变，但 CATE 差异巨大

# CART 会选择 A（因为减少了 Y 方差）
# 但我们真正需要的是 B（因为捕捉了 CATE 异质性）
```

**数学推导**:

CART 最大化:
$$\text{Gain}_{CART} = \text{Var}(Y_S) - \left[\frac{n_L}{n}\text{Var}(Y_L) + \frac{n_R}{n}\text{Var}(Y_R)\right]$$

Causal Tree 最大化:
$$\text{Gain}_{Causal} = \frac{n_L n_R}{n^2}(\hat{\tau}_L - \hat{\tau}_R)^2$$

关键洞察: CART 关心 $E[Y|X]$ 的预测，Causal Tree 关心 $E[Y(1)-Y(0)|X]$ 的异质性！

---

### 问题 2: Honest Splitting - 为什么需要？

**面试官**: 什么是 Honest Splitting？为什么需要它？直接用同一份数据不行吗？

**参考答案**:

**Honest Splitting 定义**:
将数据分成两部分：
- **Splitting Sample**: 用于决定树结构（分裂点选择）
- **Estimation Sample**: 用于估计叶节点的 CATE

**为什么需要**:

如果用同一份数据既分裂又估计，会产生三大问题：

1. **过拟合噪声**
```python
# 假设真实 CATE = 5（常数）
# 但数据中有噪声
# 分裂算法会找到"最优"分裂点，使得左右差异最大
# 但这个差异很可能只是噪声！

# 例子：
左子节点: CATE估计 = 8（实际上是 5 + 噪声）
右子节点: CATE估计 = 2（实际上是 5 - 噪声）
差异 = 6  # 看起来很大，但其实是假的！
```

2. **置信区间失效**
```python
# 标准误的计算假设：估计样本独立于模型选择
# 如果用同样的数据选择分裂，这个假设被违反
# 导致置信区间过于乐观，覆盖率低于名义水平
```

3. **选择偏差**
```python
# 分裂点是"挑选"出来的（最大化增益）
# 用同样的数据估计，会高估效应差异
```

**Honest Splitting 如何解决**:

```python
# Step 1: 用 Splitting Sample 选择分裂点
#   → 这部分数据可能过拟合噪声
split_sample → 决定树结构

# Step 2: 用独立的 Estimation Sample 估计 CATE
#   → 这部分数据没有参与分裂决策，是"诚实"的估计
estimation_sample → 估计叶节点 CATE + 置信区间

# 结果：无偏估计 + 正确的置信区间
```

**代价**:
- 有效样本量减半（一半用于分裂，一半用于估计）
- 但换来的是无偏性和正确的统计推断

**类比**:
类似于训练集/测试集划分：
- 不能用训练集评估模型（过于乐观）
- 需要独立的测试集（诚实评估）

---

### 问题 3: Causal Forest 置信区间

**面试官**: Causal Forest 如何计算置信区间？和普通随机森林有什么不同？

**参考答案**:

**普通随机森林**:
- 通常不提供置信区间
- 即使用 Bootstrap 方法，也没有理论保证

**Causal Forest 的置信区间**:

基于 Wager & Athey (2018) 的理论，Causal Forest 可以提供渐近正态的置信区间。

**关键要素**:

1. **Honest Splitting**（必需！）
```python
# 只有 Honest 版本才能保证置信区间的有效性
# 因为 Estimation Sample 独立于树结构选择
```

2. **Subsampling（子采样）**
```python
# 每棵树使用不同的子样本（不是 Bootstrap）
# 子采样比例通常是 50%
# 这样不同树之间有一定独立性
```

3. **方差估计**
```python
# 对于样本 i 的 CATE 估计
# 收集所有包含 i 的树（作为 out-of-bag 样本）
# 这些树的预测的方差 → 估计方差

V(τ̂(x_i)) = Var({τ̂_b(x_i) : i ∈ OOB_b})
```

**实现**:

```python
class HonestCausalForest:
    def predict_with_ci(self, X, alpha=0.05):
        n_samples = len(X)
        predictions = []  # 每棵树的预测

        # 收集所有树的预测
        for tree in self.trees:
            pred = tree.predict(X)
            predictions.append(pred)

        predictions = np.array(predictions)  # (n_trees, n_samples)

        # 点估计：所有树的平均
        point_estimate = predictions.mean(axis=0)

        # 方差估计：树之间预测的方差
        variance = predictions.var(axis=0, ddof=1) / self.n_trees

        # 置信区间
        z = stats.norm.ppf(1 - alpha/2)
        se = np.sqrt(variance)
        ci_lower = point_estimate - z * se
        ci_upper = point_estimate + z * se

        return point_estimate, ci_lower, ci_upper
```

**覆盖率验证**:
```python
# 在理想条件下（大样本 + 正确模型假设）
# 95% CI 应该覆盖真实 CATE 约 95% 的样本
coverage = np.mean((true_cate >= ci_lower) & (true_cate <= ci_upper))
# 应该接近 0.95
```

**与 Bootstrap 的区别**:

| 方法 | Bootstrap | Causal Forest CI |
|------|-----------|------------------|
| 理论保证 | 无（启发式） | 有（渐近正态性） |
| 需要 Honest | 不需要 | 必需 |
| 计算成本 | 高（需要重复训练） | 低（一次训练） |

---

### 问题 4: 实战场景 - CATE 与业务直觉冲突

**面试官**: 你用 Causal Forest 预测了用户的 CATE，但结果和业务团队的直觉相反。业务认为高价值用户对优惠敏感，但模型预测他们的 CATE 很小。你会怎么办？

**参考答案**:

**Step 1: 验证模型本身**

```python
# 1. 检查模型诊断指标
def validate_model(model, X, Y, T, true_cate=None):
    # a) 交叉验证性能
    cv_score = cross_val_cate_score(model, X, Y, T)

    # b) 特征重要性（是否合理？）
    feature_importance = model.feature_importances_

    # c) 预测分布（是否有异常值？）
    pred_cate = model.predict(X)
    print(f"CATE 范围: [{pred_cate.min():.2f}, {pred_cate.max():.2f}]")

    # d) 如果有真实 CATE（模拟），计算相关性
    if true_cate is not None:
        corr = np.corrcoef(pred_cate, true_cate)[0, 1]
        print(f"与真实 CATE 相关性: {corr:.3f}")

    return {'cv_score': cv_score, 'feature_importance': feature_importance}
```

**Step 2: 检查数据质量**

```python
# 2. 高价值用户的样本量足够吗？
high_value_mask = X[:, 'value_score'] > 0.8
print(f"高价值用户样本量: {high_value_mask.sum()}")
print(f"  处理组: {(high_value_mask & (T==1)).sum()}")
print(f"  控制组: {(high_value_mask & (T==0)).sum()}")

# 如果样本量不足，CATE 估计不可靠！
```

**Step 3: 理解业务逻辑**

可能的解释：

**解释A: 业务直觉可能是错的**
```
业务观察: 高价值用户收到优惠后消费更多
业务结论: 他们对优惠敏感（CATE 大）

但真相可能是：
- 高价值用户本身就消费多（高 Y(0)）
- 优惠只是锦上添花（CATE 小）
- 低价值用户才是被优惠"激活"的（CATE 大）

这是混淆了 Y(1) 和 CATE！
```

**解释B: 缺少关键特征**
```python
# 业务可能在用隐含的细分：
# "高价值但对价格敏感的用户" vs "高价值但不在意价格的用户"

# 但模型可能缺少"价格敏感度"这个特征
# 导致把两类用户混在一起，CATE 被平均掉了

# 解决：补充特征
X_new = add_feature(X, 'price_sensitivity')
```

**解释C: 处理分配偏差**
```python
# 检查倾向得分
e = estimate_propensity(X, T)

print("高价值用户的倾向得分:")
print(e[high_value_mask].describe())

# 如果 e ≈ 1（几乎都被处理）
# 缺少未处理的对照组 → CATE 估计不准
```

**Step 4: 与业务沟通**

**错误做法**:
```
"模型说你们错了，高价值用户 CATE 就是小。"
```

**正确做法**:
```
"我们的模型发现了一个有趣的现象。让我们一起深入分析：

1. 数据层面：
   - 高价值用户中，有多少收到过优惠？
   - 有没有未收到优惠的高价值用户作为对照？

2. 定义层面：
   - 你们说的'对优惠敏感'是指：
     a) 收到优惠后消费多（高 Y(1)）？
     b) 还是相对不发优惠提升大（高 CATE）？

3. 细分层面：
   - 高价值用户内部是否还有细分？
   - 是否有隐含的特征我们还没有捕捉？

4. 验证方案：
   - 我们可以做一个小规模 A/B 测试验证模型预测
   - 特别针对高价值用户这个群体"
```

**Step 5: 后续行动**

```python
# 方案A: 如果样本不足 → 补充数据
run_targeted_experiment(segment='high_value_users')

# 方案B: 如果特征不足 → 补充特征
X_augmented = engineer_features(X, business_knowledge)

# 方案C: 如果模型过简单 → 换模型
# Causal Forest → Neural Network (DRNet)

# 方案D: 如果业务理解有偏 → 教育业务
visualize_cate_vs_y1(high_value_users)
```

**关键启示**:
- 模型和直觉冲突时，两者都可能错
- 不要急于下结论，用数据和逻辑分析
- 与业务深度沟通，建立信任
- 设计验证方案，用实验说话